# Objective: For each stock: have a csv file with date,close price, and the rank of market cap

## Step 1: Download Fundamental Data

We are going to do it only for 10 stocks, because of memory limitations on google colab. Locally on a computer can be done for 545 stocks.

In [0]:
stocks=['FB','AAPL','MSFT','GOOG','JPM','V','AIG','TSLA','BAC']

The fundamental data can be found on http://www.stockpup.com/data.
Using inspect element we can find a link to download the files from Python. To download all 545 stocks we can have a list of 3000 stocks and 'try' to find which ones are his website.

In [0]:
import pandas as pd
import os

In [0]:
#os.mkdir('Fundamental_Data')

In [0]:
path='http://www.stockpup.com/data/{}_quarterly_financial_data.csv'
for stock in stocks:
  try: # in this case it is not needed. But if we are going for 545 stocks, some stocks might not work.
    df=pd.read_csv(path.format(stock))
    save=pd.DataFrame()
    save['Quarter end']=[i for i in df['Quarter end']]
    save['Shares']=[i for i in df['Shares']]
    save.to_csv('Fundamental_Data\{}.csv'.format(stock),index=False)
  except:
    pass

In [74]:
pd.read_csv('Fundamental_Data\\AAPL.csv').head(3) # Example

,Quarter end,Shares
0,2019-09-28,4443265000
1,2019-06-29,4519180000
2,2019-03-30,4601075000


## Step 2: Find the market cap for each stock and each month

We want all dates after 2005. Before that it becomes too old. The code below we make a list with all months in those years, in the format '2020-01-31'.

In [75]:
dates=['2004-12']
for i in range(2005,2020):
    for j in range(1,13):
        if j<10:
            dates.append(str(i)+'-0'+str(j))
        else:
            dates.append(str(i)+'-'+str(j))

dates[:5] #Example

['2004-12', '2005-01', '2005-02', '2005-03', '2005-04']

In [0]:
path_to_data='Fundamental_Data\{}.csv'
import datetime
# pip install yfinance
import yfinance as yf

Now the important script.

In [0]:
for stock in stocks:
    df=pd.read_csv(path_to_data.format(stock))
    quarters=[i[:7] for i in df['Quarter end']]

    first_date=df['Quarter end'][df.shape[0]-1]

    if datetime.date(int(first_date.split('-')[0]),int(first_date.split('-')[1]),int(first_date.split('-')[2]))<datetime.date(2004,12,30):
        help_dates=dates
    else:
        help_dates=dates[dates.index('{}-{}'.format(first_date.split('-')[0],first_date.split('-')[1])):]
    
    shares=[]
    for date in help_dates:
        if date in quarters:
            shares.append(df['Shares'][quarters.index(date)])
        else:
            if len(shares) != 0:
                shares.append(shares[-1])
            else:
                for i in quarters:
                    if datetime.date(int(i.split('-')[0]),int(i.split('-')[1]),1)<datetime.date(int(date.split('-')[0]),int(date.split('-')[1]),2):
                        shares.append(df['Shares'][quarters.index(i)])
                        break

    df=yf.download(stock,start='2004-11-30',progress=False)
    stock_dates=[str(i).split()[0] for i in df.index]
    stock_closes=[i for i in df['Close']]
    
    prices=[]
    for date in help_dates:
        closes=[]
        for j in stock_dates:
            if date in j:
                closes.append(stock_closes[stock_dates.index(j)])
        try:
            prices.append(sum(closes)/len(closes))
        except:
            prices.append(0)
            
    market_cap=[prices[i]*shares[i] for i in range(len(shares))]
    

    save_df=pd.DataFrame()
    save_df['Date']=help_dates
    save_df['Number of shares']=shares
    save_df['Average price']=prices
    save_df['Market cap']=market_cap

    save_df.to_csv(path_to_data.format(stock),index=False)

In [78]:
pd.read_csv('Fundamental_Data\AAPL.csv').head(15) #Example

,Date,Number of shares,Average price,Market cap
0,2004-12,408585970,4.607045,1.882374e+09
1,2005-01,408585970,4.944893,2.020414e+09
2,2005-02,408585970,5.980865,2.443697e+09
3,2005-03,823933916,5.999675,4.943336e+09
4,2005-04,823933916,5.532993,4.558821e+09
5,2005-05,823933916,5.331701,4.392969e+09
6,2005-06,829829296,5.370130,4.456291e+09
7,2005-07,829829296,5.844429,4.849878e+09
8,2005-08,829829296,6.437888,5.342348e+09
9,2005-09,842767948,7.287823,6.141944e+09


It worked.

## Step 3: Rank them.

We are going to create a dictionary, that will store stocks as keys and rank of market cap as values.

In [0]:
everything_saved={}
for stock in stocks:
    everything_saved[stock]=[]

Now we make the script to store data into that dictionary.

In [0]:
for date in dates:
    stocks_of_that_date=[]
    market_caps_of_that_date=[]
    for stock in stocks:
        df=pd.read_csv(path_to_data.format(stock))
        try:
            help_list=[i for i in df['Date']]
            m=df['Market cap'][help_list.index(date)]
            market_caps_of_that_date.append(m)
            stocks_of_that_date.append(stock)
        except:
            pass
        
    test=pd.DataFrame()
    test['Ticker']=stocks_of_that_date
    test['Market Cap']=market_caps_of_that_date
    test.sort_values(by=['Market Cap'],inplace=True,ascending=False)
    test.reset_index(drop=True,inplace=True)

    final=[i for i in test['Ticker']]
    
    for stock in stocks:
        try:
            everything_saved[stock].append(final.index(stock))
        except:
            pass

So now everything is stored in that dictionary. Now we would like to move it to that csv files above.

In [0]:
for stock in stocks:
    df=pd.read_csv(path_to_data.format(stock))
    try:
        df['Market cap rank']=everything_saved[stock]
        df.to_csv(path_to_data.format(stock),index=False)
    except:
        print(stock)

Keep in mind that the Market cap rank compares the stock to the other 9, not all the equities. That is why the market cap rank is so high!

In [82]:
pd.read_csv('Fundamental_Data\AAPL.csv').head(15) #Example

,Date,Number of shares,Average price,Market cap,Market cap rank
0,2004-12,408585970,4.607045,1.882374e+09,5
1,2005-01,408585970,4.944893,2.020414e+09,5
2,2005-02,408585970,5.980865,2.443697e+09,5
3,2005-03,823933916,5.999675,4.943336e+09,5
4,2005-04,823933916,5.532993,4.558821e+09,5
5,2005-05,823933916,5.331701,4.392969e+09,5
6,2005-06,829829296,5.370130,4.456291e+09,5
7,2005-07,829829296,5.844429,4.849878e+09,5
8,2005-08,829829296,6.437888,5.342348e+09,5
9,2005-09,842767948,7.287823,6.141944e+09,5


## Step 4: Create final file

In [0]:
for stock in stocks:
    df=pd.read_csv(path_to_data.format(stock))
    stock_data=yf.download(stock,
                           start='{}-02'.format(df['Date'][0]),
                           end='2019-12-31',
                           progress=False)
    dates=[str(i)[:10] for i in stock_data.index]
    closes=[i for i in stock_data['Close']]
    help_list=[i for i in df['Date']]
    ranks=[df['Market cap rank'][help_list.index(i[:7])] for i in dates]

    save=pd.DataFrame()
    save['Date']=dates
    save['Close']=closes
    save['Rank']=ranks
    save.to_csv('Data\{}.csv'.format(stock),index=False)

In [88]:
pd.read_csv('Data\\AAPL.csv').tail(5)

,Date,Close,Rank
3790,2019-12-23,284.000000,0
3791,2019-12-24,284.269989,0
3792,2019-12-26,289.910004,0
3793,2019-12-27,289.799988,0
3794,2019-12-30,291.519989,0
